<a href="https://colab.research.google.com/github/NikolayWTF/Hackathon-Perm/blob/main/%D0%A5%D0%B0%D0%BA%D0%B0%D1%82%D0%BE%D0%BD_%D0%9F%D0%B5%D1%80%D0%BC%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1AIIIKnNWGWTxIRdEw1OzWNeIs7h6QOdV # yolov8s_180epochs.pt
!gdown 1X9ZySLYXzoidGfGc5XThHu8X1xaz4t7P # yolov8n_150epochs.pt
!gdown 1uvsxiiGY8xHatb0HnxxnHznMgQDD2Txc # yolov8n_20epochs.pt
!gdown 1IG8jDibA-zy5v-8s_yFnZBnEHc6_sxTD # s_5.pt
!gdown 1JSFVsos9kUgOYesKEK-YM-qnzUIWSLig # yolov8m_2000epochs.pt
!gdown 1SLSp9FD8VBOkLY0gGoAk0TFsLc7StvYD # видео

In [ ]:
!pip install unrar
!pip install ultralytics

In [ ]:
#Распаковываю архив с фреймами и добавляю в папку video0
!mkdir "video0"
!unrar x -Y "video0.rar" "video0"

In [4]:
import ultralytics
import os
import math

In [ ]:
#Вычисляю M1

def count_RMSD_sum(key):
  return (ans_TBO[key] - local_TBO[key])**2

model = ultralytics.YOLO("yolov8m_2000epochs.pt")
path = "video0/frames_rgb_0/"

RMSD_w, RMSD_p, RMSD_g, RMSD_m = 0, 0, 0, 0
T = len(os.listdir(path))

for file in os.listdir(path):
    file_path = path + file
    result = model(file_path)

    # Сколько на фрейме ТБО и какие они
    file_ans = open("video0/frames_output/" + file[:-3] + "txt", "r")
    lines = file_ans.readlines()
    ans_TBO = {
        "wood": int(lines[0]),
        "glass": int(lines[1]),
        "plastic": int(lines[2]),
        "metal": int(lines[3]),
    }
    file_ans.close

    local_TBO = {
        "wood": 0,
        "glass": 0,
        "plastic": 0,
        "metal": 0,
    }

    for r in result: # Перебираем по 1 распознаному объекту
        boxes = r.boxes
        for box in boxes:
            name = model.names[int(box.cls)]
            local_TBO[name] += 1

    RMSD_w += count_RMSD_sum("wood")
    RMSD_g += count_RMSD_sum("glass")
    RMSD_p += count_RMSD_sum("plastic")
    RMSD_m += count_RMSD_sum("metal")

RMSD_w = math.sqrt(RMSD_w/T)
RMSD_m = math.sqrt(RMSD_m/T)
RMSD_g = math.sqrt(RMSD_g/T)
RMSD_p = math.sqrt(RMSD_p/T)
M1 =(RMSD_w + RMSD_p + RMSD_g + RMSD_m)/4

In [6]:
M1

0.949910625570183

In [ ]:
# Вычисление дублирующего коэффициента и M2

model = ultralytics.YOLO("yolov8n_150epochs.pt")
path = "video0/frames_rgb_0/"

valid = ["wood", "glass", "plastic", "metal"]

global_TBO = {
        "wood": 0,
        "glass": 0,
        "plastic": 0,
        "metal": 0,
          }

prev_TBO = {
        "wood": 0,
        "glass": 0,
        "plastic": 0,
        "metal": 0,
          }

lines = [100, 200, 300, 400, 500, 600] # Уровень для линии - 100, 200, 300, 400, 500, 600
T = len(os.listdir(path))
for line in lines:
    for file_ind, file in enumerate(os.listdir(path)):
        file_path = path + file
        result = model(file_path)

        for r in result:
            local_TBO = {
            "wood": 0,
            "glass": 0,
            "plastic": 0,
            "metal": 0,
            }
            boxes = r.boxes
            for box in boxes:
                x_0 = int(box.xyxy[0][0])
                x_1 = int(box.xyxy[0][2])
                name = model.names[int(box.cls)]
                if name in valid: # Если распознали ТБО
                    if file_ind == 0: # Если первый фрейм, то всё что после линии добавляем к ответу
                        if x_0 > line:
                            global_TBO[name] += 1
                        elif (x_0 < line) and (x_1 > line):
                            prev_TBO[name] += 1
                    elif file_ind == T-1: # Если последний фрейм, то всё что перед или на линии добавляем к ответу
                        if (x_0 <= line):
                            global_TBO[name] += 1
                    else:  # Все остальные варианты
                      if (x_0 < line) and (x_1 > line):
                        local_TBO[name] += 1
                        for key in valid:
                          global_TBO[key] += max(prev_TBO[key] - local_TBO[key], 0)
                          prev_TBO[key] = local_TBO[key]


In [23]:
ans = {
    "wood" : 67,
    "glass" : 59,
    "plastic" : 57,
    "metal" : 61
}

koeff = {}

for key in valid:
  global_TBO[key]/=6
  koeff[key] = ans[key] / global_TBO[key]
print(global_TBO)


{'wood': 141.83333333333334, 'glass': 125.5, 'plastic': 205.83333333333334, 'metal': 65.83333333333333}


In [24]:
koeff # Дублирующие коэффициенты

{'wood': 0.472385428907168,
 'glass': 0.4701195219123506,
 'plastic': 0.2769230769230769,
 'metal': 0.9265822784810127}